In [1]:
import os

In [2]:
os.chdir("../")

In [24]:
from dataclasses import dataclass
from pathlib import Path 


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path 
    unzip_dir: Path 

In [25]:
from Kidney_Disease_Classification_App.constants import *
from Kidney_Disease_Classification_App.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir, 
            source_url = config.source_url, 
            local_data_file = config.local_data_file, 
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


In [27]:
import os
import zipfile
import gdown
from Kidney_Disease_Classification_App import logger
from Kidney_Disease_Classification_App.utils.common import get_size

In [30]:
class DataIngestion:

    def __init__(self,config: DataIngestionConfig):
        self.config = config 
    
    def download_file(self) -> str:
        '''
        Fetch data from url
        '''
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            download_url = f"https://drive.google.com/uc?/export=download&id={file_id}" 
            gdown.download(download_url,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        '''
        zip_fle_pat: str
        extracts the zip file into the data directory 
        Function returns None 
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        


    

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-01 13:40:02,043 : INFO : common : yaml file : config/config.yaml loaded successfully]
[2024-07-01 13:40:02,044 : INFO : common : yaml file : params.yaml loaded successfully]
[2024-07-01 13:40:02,045 : INFO : common : created directory at: artifacts]
[2024-07-01 13:40:02,045 : INFO : common : created directory at: artifacts/data_ingestion]
[2024-07-01 13:40:02,045 : INFO : 673410123 : downloading data from https://drive.google.com/file/d/1LSQGDNqTZZhzL0NquYK9Kh5zY7kzZGIq/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1LSQGDNqTZZhzL0NquYK9Kh5zY7kzZGIq
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1LSQGDNqTZZhzL0NquYK9Kh5zY7kzZGIq&confirm=t&uuid=d9773d86-cfc2-4757-8d21-8dd7e9f25af8
To: /home/prem/My_Projects/End-to-End-Kidney-Disease-Classification/artifacts/data_ingestion/data.zip
100%|██████████| 1.63G/1.63G [03:04<00:00, 8.83MB/s]

[2024-07-01 13:43:09,093 : INFO : 673410123 : Downloaded data from https://drive.google.com/file/d/1LSQGDNqTZZhzL0NquYK9Kh5zY7kzZGIq/view?usp=sharing into file artifacts/data_ingestion/data.zip]
